In [ ]:
### Don't mind about this
import warnings
warnings.filterwarnings('ignore')
###

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import math
import time


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
df = pd.read_csv('/content/drive/Shareddrives/IML_Covid/Milestone_3_predataset/Covid.csv')

Mounted at /content/drive/


In [ ]:
df
for col in range(df.shape[1]):
    # col_num = 2
    print(df.columns[col])
    print(df.iloc[:,col].unique())

USMER
[0 1]
SEX
[1 0]
PATIENT_TYPE
[1 0]
DEAD
[1 0]
PNEUMONIA
[1 0]
AGE
[ 65  72  55  53  68  40  64  37  25  38  24  30  48  23  80  61  54  59
  45  26  32  49  39  27  57  20  56  47  50  46  43  28  33  16  62  58
  36  44  66  52  51  35  19  90  34  22  29  14  31  42  15   1   0  17
  41   2  10  12  13   4   7   6   8  60   5  63  75  81  67  18  70  88
  85  92  73  74  78  76  82  77  86  71  95  87  83  84  79  69  89   3
  97  93 100  91  21 103  11   9  94  96 101 107 102  98  99 109 119 116
 105 104 111 114 120 106 108 110 118 117 121 115 113]
PREGNANT
[0 1]
DIABETES
[0 1]
COPD
[0 1]
ASTHMA
[0 1]
INMSUPR
[0 1]
HIPERTENSION
[1 0]
OTHER_DISEASE
[0 1]
CARDIOVASCULAR
[0 1]
OBESITY
[0 1]
RENAL_CHRONIC
[0 1]
TOBACCO
[0 1]
MEDICAL_UNIT_1
[1 0]
MEDICAL_UNIT_2
[0 1]
MEDICAL_UNIT_3
[0 1]
MEDICAL_UNIT_4
[0 1]
MEDICAL_UNIT_5
[0 1]
MEDICAL_UNIT_6
[0 1]
MEDICAL_UNIT_7
[0 1]
MEDICAL_UNIT_8
[0 1]
MEDICAL_UNIT_9
[0 1]
MEDICAL_UNIT_10
[0 1]
MEDICAL_UNIT_11
[0 1]
MEDICAL_UNIT_12
[0 1]
MEDIC

# Decision Tree


In [ ]:
# Define the age ranges
age_ranges = [(0, 2), (3, 12), (13, 19), (20, 39), (40, 59), (60, 79), (80, float('inf'))]
# age_labels = ['Infant/Toddler', 'Child', 'Teenager', 'Young Adult', 'Middle-Aged Adult', 'Older Adult', 'Elderly']
age_labels = [0, 3, 13, 20, 40, 60, 80]

# Replace values in the 'AGE' column with age ranges
df['AGE'] = pd.cut(df['AGE'], bins=[age[0] for age in age_ranges] + [float('inf')], labels=age_labels, right=False)

# Check the updated DataFrame
print(df.head())

   USMER  SEX  PATIENT_TYPE  DEAD  PNEUMONIA AGE  PREGNANT  DIABETES  COPD  \
0      0    1             1     1          1  60         0         0     0   
1      0    0             1     1          1  60         0         0     0   
2      0    0             0     1          0  40         0         1     0   
3      0    1             1     1          0  40         0         0     0   
4      0    0             1     1          0  60         0         1     0   

   ASTHMA  ...  MEDICAL_UNIT_5  MEDICAL_UNIT_6  MEDICAL_UNIT_7  \
0       0  ...               0               0               0   
1       0  ...               0               0               0   
2       0  ...               0               0               0   
3       0  ...               0               0               0   
4       0  ...               0               0               0   

   MEDICAL_UNIT_8  MEDICAL_UNIT_9  MEDICAL_UNIT_10  MEDICAL_UNIT_11  \
0               0               0                0             

In [ ]:
df
for col in range(df.shape[1]):
    # col_num = 2
    print(df.columns[col])
    print(df.iloc[:,col].unique())

USMER
[0 1]
SEX
[1 0]
PATIENT_TYPE
[1 0]
DEAD
[1 0]
PNEUMONIA
[1 0]
AGE
[60, 40, 20, 80, 13, 0, 3]
Categories (7, int64): [0 < 3 < 13 < 20 < 40 < 60 < 80]
PREGNANT
[0 1]
DIABETES
[0 1]
COPD
[0 1]
ASTHMA
[0 1]
INMSUPR
[0 1]
HIPERTENSION
[1 0]
OTHER_DISEASE
[0 1]
CARDIOVASCULAR
[0 1]
OBESITY
[0 1]
RENAL_CHRONIC
[0 1]
TOBACCO
[0 1]
MEDICAL_UNIT_1
[1 0]
MEDICAL_UNIT_2
[0 1]
MEDICAL_UNIT_3
[0 1]
MEDICAL_UNIT_4
[0 1]
MEDICAL_UNIT_5
[0 1]
MEDICAL_UNIT_6
[0 1]
MEDICAL_UNIT_7
[0 1]
MEDICAL_UNIT_8
[0 1]
MEDICAL_UNIT_9
[0 1]
MEDICAL_UNIT_10
[0 1]
MEDICAL_UNIT_11
[0 1]
MEDICAL_UNIT_12
[0 1]
MEDICAL_UNIT_13
[0 1]
CLASIFFICATION_FINAL
[1 0]


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer, confusion_matrix

training_data = df.drop('CLASIFFICATION_FINAL', axis=1)
training_label = df['CLASIFFICATION_FINAL']

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc_score': make_scorer(roc_auc_score),
}

In [ ]:
folds = 10
ID3_model = DecisionTreeClassifier(criterion='entropy', random_state = 42)
ID3_scores = cross_validate(ID3_model, training_data, training_label, cv=folds, scoring=scoring_metrics, error_score='raise')

In [ ]:

for i in range(folds):
    print(f"Accuracy for fold {i} = {ID3_scores['test_accuracy'][i]}")
    print(f"Precision for fold {i} = {ID3_scores['test_precision'][i]}")
    print(f"Recall for fold {i} = {ID3_scores['test_recall'][i]}")
    print(f"F1-score for fold {i} = {ID3_scores['test_f1'][i]}")
    print(f"AUC for fold {i} = {ID3_scores['test_roc_auc_score'][i]}")
    print("\n")


print("\nFinal Metrics>>")
print(f"Accuracy = {sum(ID3_scores['test_accuracy'])/folds}")
print(f"Precision = {sum(ID3_scores['test_precision'])/folds}")
print(f"Recall = {sum(ID3_scores['test_recall'])/folds}")
print(f"F1-score = {sum(ID3_scores['test_f1'])/folds}")
print(f"AUC = {sum(ID3_scores['test_roc_auc_score'])/folds}")

Accuracy for fold 0 = 0.6947204791241488
Precision for fold 0 = 0.5925706483938278
Recall for fold 0 = 0.5868411653655798
F1-score for fold 0 = 0.5896919902072625
AUC for fold 0 = 0.6729819594723117


Accuracy for fold 1 = 0.6359457552118103
Precision for fold 1 = 0.5325782641893612
Recall for fold 1 = 0.2135313026174805
F1-score for fold 1 = 0.30484029573514954
AUC for fold 1 = 0.550825962000181


Accuracy for fold 2 = 0.6280779721146693
Precision for fold 2 = 0.5061129200712662
Recall for fold 2 = 0.2101637838665238
F1-score for fold 2 = 0.2969986480396575
AUC for fold 2 = 0.5438650171236366


Accuracy for fold 3 = 0.3248297697838982
Precision for fold 3 = 0.21339815335498033
Recall for fold 3 = 0.3001173529261697
F1-score for fold 3 = 0.249435462496687
AUC for fold 3 = 0.31985002583865596


Accuracy for fold 4 = 0.650041007839173
Precision for fold 4 = 0.5589204973624717
Recall for fold 4 = 0.3027450380121435
F1-score for fold 4 = 0.3927519443984776
AUC for fold 4 = 0.58005817237189

# Decision Tree

In [ ]:
folds = 10
C45_model = DecisionTreeClassifier(criterion='gini', splitter='best', random_state=42)
C45_scores = cross_validate(C45_model, training_data, training_label, cv=folds, scoring=scoring_metrics, error_score='raise')

In [ ]:

for i in range(folds):
    print(f"Accuracy for fold {i} = {C45_scores['test_accuracy'][i]}")
    print(f"Precision for fold {i} = {C45_scores['test_precision'][i]}")
    print(f"Recall for fold {i} = {C45_scores['test_recall'][i]}")
    print(f"F1-score for fold {i} = {C45_scores['test_f1'][i]}")
    print(f"AUC for fold {i} = {C45_scores['test_roc_auc_score'][i]}")
    print("\n")


print("\nFinal Metrics>>")
print(f"Accuracy = {sum(C45_scores['test_accuracy'])/folds}")
print(f"Precision = {sum(C45_scores['test_precision'])/folds}")
print(f"Recall = {sum(C45_scores['test_recall'])/folds}")
print(f"F1-score = {sum(C45_scores['test_f1'])/folds}")
print(f"AUC = {sum(C45_scores['test_roc_auc_score'])/folds}")

Accuracy for fold 0 = 0.7054015907226916
Precision for fold 0 = 0.6069359697227157
Recall for fold 0 = 0.6014082351140364
F1-score for fold 0 = 0.6041594587321724
AUC for fold 0 = 0.6844461218214105


Accuracy for fold 1 = 0.6358503881439661
Precision for fold 1 = 0.5323053396202371
Recall for fold 1 = 0.21312311852645544
F1-score for fold 1 = 0.3043795088537492
AUC for fold 1 = 0.5506675598724267


Accuracy for fold 2 = 0.6281828758892979
Precision for fold 2 = 0.5064440898490242
Recall for fold 2 = 0.2105209449461707
F1-score for fold 2 = 0.29741223960210483
AUC for fold 2 = 0.5440207527045811


Accuracy for fold 3 = 0.3296744168303801
Precision for fold 3 = 0.22100181266713328
Recall for fold 3 = 0.31414868105515587
F1-score for fold 3 = 0.25946879905601733
AUC for fold 3 = 0.32654586047884204


Accuracy for fold 4 = 0.6499551774781133
Precision for fold 4 = 0.5588221414751545
Recall for fold 4 = 0.3021072503699168
F1-score for fold 4 = 0.39219063074400967
AUC for fold 4 = 0.5798611

# Naive Bayes

In [ ]:
folds = 10
NV_model = GaussianNB()
NV_scores = cross_validate(NV_model, training_data, training_label, cv=folds, scoring=scoring_metrics, error_score='raise')

In [ ]:

for i in range(folds):
    print(f"Accuracy for fold {i} = {C45_scores['test_accuracy'][i]}")
    print(f"Precision for fold {i} = {C45_scores['test_precision'][i]}")
    print(f"Recall for fold {i} = {C45_scores['test_recall'][i]}")
    print(f"F1-score for fold {i} = {C45_scores['test_f1'][i]}")
    print(f"AUC for fold {i} = {C45_scores['test_roc_auc_score'][i]}")
    print("\n")


print("\nFinal Metrics>>")
print(f"Accuracy = {sum(C45_scores['test_accuracy'])/folds}")
print(f"Precision = {sum(C45_scores['test_precision'])/folds}")
print(f"Recall = {sum(C45_scores['test_recall'])/folds}")
print(f"F1-score = {sum(C45_scores['test_f1'])/folds}")
print(f"AUC = {sum(C45_scores['test_roc_auc_score'])/folds}")

Accuracy for fold 0 = 0.7054015907226916
Precision for fold 0 = 0.6069359697227157
Recall for fold 0 = 0.6014082351140364
F1-score for fold 0 = 0.6041594587321724
AUC for fold 0 = 0.6844461218214105


Accuracy for fold 1 = 0.6358503881439661
Precision for fold 1 = 0.5323053396202371
Recall for fold 1 = 0.21312311852645544
F1-score for fold 1 = 0.3043795088537492
AUC for fold 1 = 0.5506675598724267


Accuracy for fold 2 = 0.6281828758892979
Precision for fold 2 = 0.5064440898490242
Recall for fold 2 = 0.2105209449461707
F1-score for fold 2 = 0.29741223960210483
AUC for fold 2 = 0.5440207527045811


Accuracy for fold 3 = 0.3296744168303801
Precision for fold 3 = 0.22100181266713328
Recall for fold 3 = 0.31414868105515587
F1-score for fold 3 = 0.25946879905601733
AUC for fold 3 = 0.32654586047884204


Accuracy for fold 4 = 0.6499551774781133
Precision for fold 4 = 0.5588221414751545
Recall for fold 4 = 0.3021072503699168
F1-score for fold 4 = 0.39219063074400967
AUC for fold 4 = 0.5798611